In [2]:
import pandas as pd

def capacitated_lot_sizing(requirements, setup_cost, holding_cost, initial_inventory, production_capacity):
    periods = len(requirements)
    production_plan = requirements.copy()
    ending_inventory = initial_inventory
    total_cost = 0
    detailed_plan = []

    # Initial Lot-for-Lot production plan
    for i in range(periods):
        production_amount = requirements[i]
        ending_inventory += production_amount - requirements[i]
        detailed_plan.append((i + 1, requirements[i], production_amount, ending_inventory))

    # Adjust production to meet capacity constraints
    for i in range(periods):
        if production_plan[i] > production_capacity:
            excess = production_plan[i] - production_capacity
            production_plan[i] = production_capacity

            # Shift excess to previous periods
            for j in range(i - 1, -1, -1):
                available_capacity = production_capacity - production_plan[j]
                if available_capacity >= excess:
                    production_plan[j] += excess
                    excess = 0
                    break
                else:
                    production_plan[j] += available_capacity
                    excess -= available_capacity

    # Recalculate the ending inventory and cost with the adjusted plan
    ending_inventory = initial_inventory
    for i in range(periods):
        production_amount = production_plan[i]
        inventory_cost = ending_inventory * holding_cost
        total_cost += inventory_cost + (setup_cost if production_amount > 0 else 0)
        ending_inventory += production_amount - requirements[i]
        detailed_plan[i] = (i + 1, requirements[i], production_amount, ending_inventory)

    df = pd.DataFrame(detailed_plan, columns=['Period', 'Requirements', 'Production', 'Ending Inventory'])
    return df, total_cost

# Example usage
requirements = [85, 60, 40, 105, 120, 75]
setup_cost = 300
holding_cost = 0.5
initial_inventory = 0
production_capacity = 100

print("Capacitated Lot Sizing Heuristic:")
df, cost = capacitated_lot_sizing(requirements, setup_cost, holding_cost, initial_inventory, production_capacity)
print(df.to_string(index=False))
print("Total Cost: ", cost)


Capacitated Lot Sizing Heuristic:
 Period  Requirements  Production  Ending Inventory
      1            85          85                 0
      2            60          60                 0
      3            40          65                25
      4           105         100                20
      5           120         100                 0
      6            75          75                 0
Total Cost:  1822.5
